In [2]:
#import modules
from lxml import html
import re
import requests
import pandas as pd
from datetime import datetime

#get html and tree
html_page_link = 'candidateEvalData/webpage.html'
with open(html_page_link) as f:
    html_content = f.read()
utf8_parser = html.HTMLParser(encoding='utf-8')
html_tree = html.document_fromstring(html_content , parser=utf8_parser)

data = {}
# parse artist name
artist_name = html_tree.xpath("//h1[@class='lotName']")[0]
data["Artist name"] = artist_name.text.split(" (")[0]

# parse painting name
painting_name = html_tree.xpath("//h2[@class='itemName']/i")[0]
data["Painting name"] = painting_name.text

# parse price GBP
price_GBP = html_tree.xpath("//span[@id='main_center_0_lblPriceRealizedPrimary']")[0]
data["Price GBP"] = int(re.sub(r'(,)|([A-Z])*', "", price_GBP.text))

# parse price US
price_US = html_tree.xpath("//div[@id='main_center_0_lblPriceRealizedSecondary']")[0]
data["Price USD"] = int(re.sub(r'(,)|([A-Z])*', "", price_US.text))

# parse price GBP est
price_GBP_est = html_tree.xpath("//span[@id='main_center_0_lblPriceEstimatedPrimary']")[0]
price_GBP_sub = re.sub(r'(,)|([A-Z])*|(\s)', "", price_GBP_est.text)
est_GBP_split = price_GBP_sub.split("-")
data["Price GBP est"] = [(int(est_GBP_split[0]),int(est_GBP_split[1]))]

# parse price US est
price_US_est = html_tree.xpath("//span[@id='main_center_0_lblPriceEstimatedSecondary']")[0]
price_US_sub = re.sub(r'(,)|([A-Z])*|(\s)|[()]', "", price_US_est.text)
est_US_split = price_US_sub.split("-")
data["Price US est"] = [(int(est_US_split[0]),int(est_US_split[1]))]

# image link
image_link = html_tree.xpath("//img[@id='imgLotImage']")[0]
data["Image link"] = image_link.attrib['src']

df = pd.DataFrame(data=data)
df.head()

,Artist name,Painting name,Price GBP,Price USD,Price GBP est,Price US est,Image link
0,Peter Doig,The Architect's Home in the Ravine,11282500,16370908,"(10000000, 15000000)","(14509999, 21764999)",http://www.christies.com/lotfinderimages/D5973...


In [8]:
import pandas as pd
dim_df = pd.read_csv("candidateEvalData/dim_df_correct.csv")
import pandas as pd
import re
dim_df = pd.read_csv("candidateEvalData/dim_df_correct.csv")
regular_expressions = [r"(\d+(\.|,)+\d*)",r"(\d+(\.|,)*\d*)"]
# used two regular expression for collecting centimeters  dimensions
for row in dim_df["rawDim"]:
    
    for r in regular_expressions:
        matches = re.findall(r,row)
        if len(matches)  >= 2:
            print(row, " ".join(m[0] for m in matches))
            break



19×52cm 19 52
50 x 66,4 cm 50 66,4
168.9 x 274.3 x 3.8 cm (66 1/2 x 108 x 1 1/2 in.) 168.9 274.3 3.8
Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Image: 14 × 9 7/8 in. (35.6 × 25.1 cm) 41.3 31.1 35.6 25.1
5 by 5in 5 5


In [4]:
import scrapy
import pandas as pd
from scrapy import Item, Field
import re
from scrapy.crawler import CrawlerProcess


# for simplicity and performance, I iterate over the content
# of this page that contains all the artwork pieces
# as an alternative we could iterate over each page, but since 
# we can get all art items in a single pace like so, we can do it like this in
# a initial faze
URL = "https://www.bearspace.co.uk/purchase?page=1000"
items = []

class ArtPiece(Item):
    url = Field()
    title = Field()
    media = Field()
    height_cm = Field()
    width_cm = Field()
    price_gbp = Field()

class ArtCrawler(scrapy.Spider):
    name = "artcrawler"
    start_urls = [URL]
    
    def parse(self, response):
        sels = response.xpath("//a[@class='oQUvqL x5qIv3']")
        # for all art pieces in the main page we run 
        # the parsing function for each one
        for s in sels:
            url = s.attrib['href']
            yield scrapy.Request(url, callback = self.parse_item)
            
    def parse_item(self, response):
        art_piece = ArtPiece()
        
        # getting the art title and price in GBP but data-hooks
        art_piece["url"] = response.url
        art_piece["title"] = response.xpath("//h1[@data-hook='product-title']/text()").get()
        art_piece["price_gbp"] = response.xpath("//span[@data-hook='formatted-primary-price']/text()").get()
        # media and dimensions are in the 'description' data-hook container
        contents = response.xpath("//pre[@data-hook='description']//text()").extract()
        dimensions = None
        media = []
        # dimensions are retrieved using a regular expression and the first 
        # matches are captured, the rest are added to the media field
        # since the data in the content does not follow a specific format
        if contents:
            for content in contents:
                check_dim = re.findall(r"(\d+(\.|,)*\d*)", content)
                if check_dim and len(check_dim) >= 2 and not dimensions:
                    dimensions = check_dim
                elif not dimensions:
                    media.append(content)
            if dimensions:
                art_piece["height_cm"] = dimensions[0][0]
                art_piece["width_cm"] = dimensions[1][0]
            art_piece["media"] = " ".join(media)
        items.append(art_piece)
        yield art_piece
        
if __name__ == "__main__":
    
    process = CrawlerProcess()
    process.crawl(ArtCrawler)
    process.start()
    
    print(items)
    
    df = pd.DataFrame(items, columns=['url','title','media','height_cm','width_cm', 'price_gbp'])
    df

2022-12-18 13:42:14 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
2022-12-18 13:42:14 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.10.2, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.9.16 (main, Dec  7 2022, 10:06:04) - [Clang 14.0.0 (clang-1400.0.29.202)], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.3, Platform macOS-13.1-arm64-arm-64bit
2022-12-18 13:42:14 [scrapy.crawler] INFO: Overridden settings:
{}
2022-12-18 13:42:14 [py.warnings] WARNING: /opt/homebrew/lib/python3.9/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the

2022-12-18 13:42:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/purple-motion-by-dionne-cole>
{'height_cm': '59.4',
 'media': '',
 'price_gbp': '£1,200.00',
 'title': 'Purple Motion by Dionne Cole',
 'url': 'https://www.bearspace.co.uk/product-page/purple-motion-by-dionne-cole',
 'width_cm': '84.1'}
2022-12-18 13:42:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/castle-2-by-jane-ward> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/meld-5-by-vic-wright> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/unfathomable-deeps-by-jane-ward-1> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://ww

2022-12-18 13:42:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/retreating-then-advancing-repeatedly-over-the-centuries-3-by-janie-kidston>
{'height_cm': '50',
 'media': '',
 'price_gbp': '£540.00',
 'title': "'Retreating, then advancing repeatedly over the centuries 3' by "
          'Janie Kidston',
 'url': 'https://www.bearspace.co.uk/product-page/retreating-then-advancing-repeatedly-over-the-centuries-3-by-janie-kidston',
 'width_cm': '60'}
2022-12-18 13:42:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/walking-out-over-ice-in-midsummer-1>
{'height_cm': '84',
 'media': 'Archival Digital Print',
 'price_gbp': '£940.00',
 'title': 'Walking Out Over Ice in Midsummer 1 by Janie Kidston',
 'url': 'https://www.bearspace.co.uk/product-page/walking-out-over-ice-in-midsummer-1',
 'width_cm': '59'}
2022-12-18 13:42:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/

2022-12-18 13:42:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/under-a-bruised-and-leaden-sky-by-dominic-badnum> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/i-crash-into-your-arms>
{'height_cm': '70',
 'media': 'Oil with charcoal, watercolour, salt & sand on canvas',
 'price_gbp': '£1,400.00',
 'title': 'I Crash Into Your Arms by Dominic Bradnum',
 'url': 'https://www.bearspace.co.uk/product-page/i-crash-into-your-arms',
 'width_cm': '100'}
2022-12-18 13:42:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/garden-city-state-1b-by-jane-ward>
{'height_cm': '60',
 'media': '',
 'price_gbp': '£490.00',
 'title': 'Garden City State 1b by Jane Ward',
 'url': 'https://www.bearspace.co.uk/product-page/garden-city-state-1b-by-jane-ward',
 'width_cm': '45'}
2022-12-18 13:42:22 [scrapy.core.e

2022-12-18 13:42:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/myriorama-room-series-armchair-by-bella-easton>
{'height_cm': '89',
 'media': 'Artist: Bella Easton',
 'price_gbp': '£2,800.00',
 'title': 'MYRIORAMA ROOM series –ARMCHAIR by Bella Easton',
 'url': 'https://www.bearspace.co.uk/product-page/myriorama-room-series-armchair-by-bella-easton',
 'width_cm': '117'}
2022-12-18 13:42:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/myriorama-room-series-lamp-by-bella-easton> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/myriorama-room-series-fireplace-ii-by-bella-easton> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/myriorama-room-series-lamp-by-bella-easton>
{'h

...

2022-12-18 13:42:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/fragstudy-4-by-jim-cheatle>
{'height_cm': '30.5',
 'media': 'Resin, pigments, inkjet prints, MDF',
 'price_gbp': '£490.00',
 'title': 'Fragstudy 4 by Jim Cheatle',
 'url': 'https://www.bearspace.co.uk/product-page/fragstudy-4-by-jim-cheatle',
 'width_cm': '23'}
2022-12-18 13:42:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/fragstudy-2-by-jim-cheatle> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/fragstudy-2-by-jim-cheatle>
{'height_cm': '30.5',
 'media': 'Resin, pigments, inkjet prints, MDF',
 'price_gbp': '£490.00',
 'title': 'Fragstudy 2 by Jim Cheatle',
 'url': 'https://www.bearspace.co.uk/product-page/fragstudy-2-by-jim-cheatle',
 'width_cm': '23'}
2022-12-18 13:42:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET

2022-12-18 13:42:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/balancing-acts-iii-by-elise-wootten> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/all-recognisable-entities-by-elise-wootten>
{'height_cm': '40.9',
 'media': 'Set of four Beech, Mdf, Acrylic paint, Vinyl',
 'price_gbp': '£1,200.00',
 'title': 'All Recognisable Entities by Elise Wootten',
 'url': 'https://www.bearspace.co.uk/product-page/all-recognisable-entities-by-elise-wootten',
 'width_cm': '50.2'}
2022-12-18 13:42:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/to-the-zenith-s-height-by-jane-ward> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/somewhat-confusing-ii-by-elise-wootten> (referer: https

2022-12-18 13:42:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bearspace.co.uk/product-page/constructed-veneer-4-by-olly-fathers> (referer: https://www.bearspace.co.uk/purchase?page=1000)
2022-12-18 13:42:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/balancing-acts-v-by-elise-wootten>
{'height_cm': '40.7',
 'media': 'Inkjet Print (Hahnemuhle Ultra-Smooth)',
 'price_gbp': '£340.00',
 'title': 'Balancing Acts V by Elise Wootten',
 'url': 'https://www.bearspace.co.uk/product-page/balancing-acts-v-by-elise-wootten',
 'width_cm': '50.8'}
2022-12-18 13:42:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/tommy-by-olly-fathers>
{'height_cm': '100',
 'media': 'Spraypaint, board, card, digital print collage',
 'price_gbp': '£1,940.00',
 'title': 'Tommy by Olly Fathers',
 'url': 'https://www.bearspace.co.uk/product-page/tommy-by-olly-fathers',
 'width_cm': '70'}
2022-12-18 13:42:39 [scrapy.core.

2022-12-18 13:42:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.bearspace.co.uk/product-page/p8-by-olly-fathers>
{'height_cm': '30',
 'media': 'Acrylic paint on sprayed board',
 'price_gbp': '£240.00',
 'title': 'P8 by Olly Fathers',
 'url': 'https://www.bearspace.co.uk/product-page/p8-by-olly-fathers',
 'width_cm': '30'}
2022-12-18 13:42:41 [scrapy.core.engine] INFO: Closing spider (finished)
2022-12-18 13:42:41 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 95105,
 'downloader/request_count': 200,
 'downloader/request_method_count/GET': 200,
 'downloader/response_bytes': 37268353,
 'downloader/response_count': 200,
 'downloader/response_status_count/200': 200,
 'elapsed_time_seconds': 26.168133,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2022, 12, 18, 11, 42, 41, 314384),
 'httpcompression/response_bytes': 234001246,
 'httpcompression/response_count': 200,
 'item_scraped_count': 199,
 'log_count/DEBUG': 404,
 'log

[{'height_cm': '40',
 'media': 'oil with charcoal, watercolour and pencil on canvas',
 'price_gbp': '£640.00',
 'title': 'This Day Is Ours by Dominic Bradnum',
 'url': 'https://www.bearspace.co.uk/product-page/you-took-my-breath-away-by-dominic-badnum',
 'width_cm': '50'}, {'height_cm': '18',
 'media': '13K gold leaf, copper leaf, white casting cement, iron and pigment',
 'price_gbp': '£440.00',
 'title': 'Meld 4 by Vic Wright',
 'url': 'https://www.bearspace.co.uk/product-page/meld-4-by-vic-wright',
 'width_cm': '15'}, {'height_cm': '40',
 'media': 'oil on canvas with charcoal and watercolour',
 'price_gbp': '£840.00',
 'title': 'Take My Hand by Dominic Badnum',
 'url': 'https://www.bearspace.co.uk/product-page/take-my-hand-by-dominic-badnum',
 'width_cm': '50'}, {'height_cm': '80',
 'media': 'oil on canvas',
 'price_gbp': '£1,600.00',
 'title': 'The Tide Seems To Whisper Your Name by Dominic Badnum',
 'url': 'https://www.bearspace.co.uk/product-page/the-tide-seems-to-whisper-your-nam

In [5]:
import pandas as pd

flights = pd.read_csv("candidateEvalData/flights.csv")
airports = pd.read_csv("candidateEvalData/airports.csv")
weather = pd.read_csv("candidateEvalData/weather.csv")
airlines = pd.read_csv("candidateEvalData/airlines.csv")

# merge the airlines and flights dataframe and rename the 'name' column to 'airline'
df = flights.merge(airlines, on="carrier")
df.rename(columns = {'name':'airline'}, inplace = True)

# filter if string contains "FetBlue"
df = df[df['airline'].str.contains("JetBlue")]

# sort by origin
df = df.sort_values(by=['origin'])

# filter for values of more than 100
df['origin'].value_counts().loc[lambda x : x>100]

"""
INNER JOIN: Retrieves only the rows that have a match in both tables.
LEFT JOIN: Retrieves all rows from the left table and any matching rows from the right table. 
            If there is no match, NULL values are returned for right table's columns.
RIGHT JOIN: Retrieves all rows from the right table and any matching rows from the left table.
FULL OUTER JOIN: Retrieves all rows from both tables, whether or not there is a match in the other table. 
            If there is no match, NULL values are returned for the non-matching columns.

-- Add full airline name to the flights dataframe and show the arr_time, origin, dest and the name of the airline.

ALTER TABLE flights ADD airline CHAR(100)	;

UPDATE flights AS f, airlines as a
SET f.airline = a.name
WHERE f.carrier = a.carrier;

select arr_time, origin, dest, airline from flights;

-- Filter resulting data.frame to include only flights containing the word JetBlue

select * from flights
where airline like '%JetBlue%'

-- Summarise the total number of flights by origin in ascending.

select * from flights
order by origin ASC;

-- Filter resulting data.frame to return only origins with more than 100 flights.
SELECT origin, COUNT(*) FROM flights
group by flights
having COUNT(*) > 100; 

"""

JFK    148
Name: origin, dtype: int64